In [34]:
import os
import sys
import numpy as np
import torch
import plotly.graph_objects as go
import tqdm
import pickle
import kaleido
import astropy.units as u
from astroquery.ipac.irsa import Irsa
ROOT = os.path.join("./")
sys.path.append(ROOT + "lib")
from helpers import get_coordinates
from lightsource import LightSource
from plotly_style import update_layout

buckets = pickle.load(open(ROOT + "cached_data/raw_classes.pkl", "rb"))

In [2]:
def get_trace(lightsource, key):

  x_data = lightsource["norm"]["day"]
  y_data = lightsource["norm"][key]

  sort_idxs = np.argsort(x_data)

  x_data = x_data[sort_idxs]
  y_data = y_data[sort_idxs]


  return go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(size=5, opacity=.75),
    name=key,
  )

def plot_from_lightsource(lightsource, ys=["w1"]):

  fig = go.Figure()

  for y in ys:
    fig.add_trace(get_trace(lightsource, y))

  update_layout(fig, legend_out=True)

  fig.layout.width = 800
  fig.layout.height = 0.65 * fig.layout.width

  return fig

def plot_from_tensor(data):
  fig = go.Figure()

  w1 = data[:, 0].numpy()
  std = data[:, 1].numpy()


  day = data[:, -1].numpy()


  fig.add_trace(go.Scatter(x=day, y=w1, marker=dict(size=5, opacity=0.7), name="w1mpro z-scored", mode='markers'))
  fig.add_trace(go.Scatter(x=day, y=std, marker=dict(size=5, opacity=0.7), name="w2mpro z-scored", mode='markers'))

  update_layout(fig, legend_out=True)

  fig.layout.width = 800
  fig.layout.height = 0.65 * fig.layout.width


  return fig

In [ ]:
# ----------------------- Begin Work on Dataset ----------------------- # 
ex = buckets["nova"]["V569 Vul"]
flux = ex["raw"]["w1flux"]

ADJ = 0
DIV = 0.001
norm = np.arcsinh((flux - ADJ) /DIV)
tr1 = go.Scatter(
    x=ex["norm"]["day"],
    y=norm,
    marker=dict(size=5, opacity=0.7), name="normalized flux", mode='markers'
)

tr2 = go.Scatter(
    x=ex["norm"]["day"],
    y=flux,
    marker=dict(size=5, opacity=0.7), name="raw flux", mode='markers'
)

fig = plot_from_lightsource(ex, ["w1"])
fig.add_trace(tr1)
fig.add_trace(tr2)
fig.show()

print([n for n in flux])


In [36]:
c = get_coordinates("19 52 08.25	+27 42 20.9")

tbl = Irsa.query_region(c, catalog="neowiser_p1bs_psd", spatial="Cone",
                            radius=3 * u.arcsec)
tbl = tbl.to_pandas()
tbl = tbl.loc[tbl["qual_frame"]>= 4]
ex = LightSource(tbl)
plot_from_lightsource(ex)


In [29]:
tbl[]

308

In [33]:
df = ex.get_pandas()
np.mean(df["ra"]), np.mean(df["dec"])
df

,ra,dec,sigra,sigdec,sigradec,glon,glat,elon,elat,w1x,...,j_msig_2mass,h_m_2mass,h_msig_2mass,k_m_2mass,k_msig_2mass,x,y,z,spt_ind,htm20
0,298.034719,27.706375,0.0666,0.0707,0.0110,64.229114,0.329171,308.029944,47.512182,349.064,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781458,0.464941,203320020,14259834931654
1,298.034639,27.706375,0.0977,0.0903,-0.0324,64.229078,0.329232,308.029843,47.512202,83.359,...,0.0217,12.909,0.0206,12.741,0.0182,0.416115,-0.781459,0.464941,203320020,14259834931663
2,298.034713,27.706375,0.0798,0.0666,-0.0075,64.229111,0.329176,308.029936,47.512183,42.345,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781459,0.464941,203320020,14259834931654
3,298.034726,27.706401,0.0714,0.0711,-0.0173,64.229139,0.329179,308.029963,47.512204,717.716,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781458,0.464941,203320020,14259834931654
4,298.034692,27.706385,0.0608,0.0661,-0.0215,64.229110,0.329196,308.029914,47.512197,927.552,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781459,0.464941,203320020,14259834931654
5,298.034698,27.706433,0.0696,0.0810,-0.0128,64.229154,0.329217,308.029941,47.512242,468.534,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781458,0.464941,203320020,14259834931655
8,298.034644,27.706334,0.0753,0.0831,0.0229,64.229045,0.329207,308.029832,47.512160,350.402,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781459,0.464940,203320020,14259834931662
9,298.034664,27.706387,0.0690,0.0789,-0.0233,64.229099,0.329220,308.029879,47.512207,662.518,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781459,0.464941,203320020,14259834931662
10,298.034682,27.706386,0.0613,0.0638,-0.0171,64.229107,0.329205,308.029902,47.512201,284.668,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781459,0.464941,203320020,14259834931662
11,298.034671,27.706319,0.0772,0.0710,-0.0260,64.229044,0.329179,308.029860,47.512140,165.136,...,0.0217,12.909,0.0206,12.741,0.0182,0.416116,-0.781459,0.464940,203320020,14259834931662
